# Debugging and Checking emissions

#### This script is almost identical to the emission_analysis.ipynb under LOCAETA-reports (jupyter notebook). I created this script for debugging and checking emission scenarios. 

Author : Yunha Lee

Date: March 12, 2025 

# 1. Read emissions file

Here are the point-source facilities emissions where amine-based CCS technology is applied. The case presented in here is over the Louisanna State. The data in the below presents the NEI2020 emissions (as "_old") as well as modified emissions by the CCS technology (without "_old") for major air pollutants. 

In [9]:
import geopandas as gpd
import os


# read base and sens emission scenarios
gdf_emis = gpd.read_file(
    #'/Users/yunhalee/Documents/LOCAETA/RCM/INMAP/evaldata_v1.6.1/2020_nei_emissions/combined_NEI2020_pt_oilgas_ptegu_ptnonipm.shp')
 #'/Users/yunhalee/Documents/LOCAETA/CS_emissions/USA_point_CCS_reduced_emis.shp')

 '/Users/yunhalee/Documents/LOCAETA/CS_emissions/Colorado_point_CCS_reduced_emis.shp' ) #new_LA_point_CCS.shp') #)

output_dir = '/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/outputs/emissions/CO_CCS/'
national_scale_on = True # Set it False if the scenario is only for single State

if not os.path.exists(output_dir ):
    os.makedirs(output_dir )

# Reset index to ensure proper comparison
gdf_emis.reset_index(drop=True, inplace=True)

print(gdf_emis.head())
# def subset_data(final_df, state_fips=None):
#     if state_fips:
#         # Match the first two digits of FIPS with State FIPS code
#         return final_df[final_df['FIPS'].astype(str).str[:2] == str(state_fips)]
#     return final_df

# # Dictionary to loop over
# regions = {"LA": '22'} #, "Nation": None}

# final_df = gpd.GeoDataFrame(gdf_emis, geometry= "geometry")
# type(final_df)

# for region_name, state_fips in regions.items():
    
#     # Subset the DataFrame based on the chosen state or national
#     final_df_subset = subset_data(final_df, state_fips)

#     print(f"Processing data for: {region_name}")

# IMPORTANT NOTE : This subset function doesn't include total emissions by EIS_ID. Some cases,     
def subset_pollutants_with_difference(df, pollutants):
    mask = False  # Initialize a mask to filter rows
    for pollutant in pollutants:
        col_current = pollutant
        col_old = f'{pollutant}_old'
        
        if col_current in df.columns and col_old in df.columns:
            # Check where the pollutant and its "_old" version differ
            mask |= (df[col_current] != df[col_old])

    # Subset the dataframe based on the mask
    return df[mask]

pollutants = ['VOC', 'NOx', 'NH3', 'SOx', 'PM2_5']
subset_df = subset_pollutants_with_difference(gdf_emis, pollutants)

subset_df


    FIPS       SCC  VOC_old  NOx_old  NH3_old  SOx_old  PM2_5_old   height  \
0  08001  20100201   2.2000   99.300    0.000   1.8000    12.7001  22.8600   
1  08123  20100202   0.0019    0.040    0.000   0.0002     0.0073  24.3840   
2  08123  20100201   5.7767  146.700    0.000   7.7000    19.3002  53.3400   
3  08123  10100602   0.4150    7.545    0.000   0.0440     0.1446  31.0896   
4  08123  20100201  16.3000  439.500   93.159   9.2100    55.3695  53.3400   

      diam       temp  ...  PM25CON_re       NOx       SOx  PM25FIL_ou  \
0  5.54736  666.66667  ...    6.482381   71.4960  0.018000    0.174119   
1  1.06680  180.55556  ...    0.003115    0.0288  0.000002    0.000120   
2  5.63880   89.44444  ...    9.851209  105.6240  0.077000    0.264606   
3  1.06680  333.33333  ...    0.085360    5.4324  0.000440    0.001614   
4  4.57200  100.00000  ...   28.261692  316.4400  0.092100    0.759116   

   PM25CON_ou     PM2_5  PM25_reduc        VOC         NH3  \
0    0.413769  0.587888 

,FIPS,SCC,VOC_old,NOx_old,NH3_old,SOx_old,PM2_5_old,height,diam,temp,...,PM25CON_re,NOx,SOx,PM25FIL_ou,PM25CON_ou,PM2_5,PM25_reduc,VOC,NH3,geometry
0,08001,20100201,2.2000,99.3000,0.000,1.8000,12.7001,22.8600,5.54736,666.66667,...,6.482381,71.496000,0.018000,0.174119,0.413769,0.587888,12.112213,2.494518,28.113070,POINT (-652329.196 -711.107)
1,08123,20100202,0.0019,0.0400,0.000,0.0002,0.0073,24.3840,1.06680,180.55556,...,0.003115,0.028800,0.000002,0.000120,0.000199,0.000318,0.006982,1.837425,175.209220,POINT (-641913.733 36890.849)
2,08123,20100201,5.7767,146.7000,0.000,7.7000,19.3002,53.3400,5.63880,89.44444,...,9.851209,105.624000,0.077000,0.264606,0.628801,0.893406,18.406794,7.612225,175.209220,POINT (-641913.733 36890.849)
3,08123,10100602,0.4150,7.5450,0.000,0.0440,0.1446,31.0896,1.06680,333.33333,...,0.085360,5.432400,0.000440,0.001614,0.005449,0.007062,0.137538,3.097226,256.030706,POINT (-663965.526 55833.920)
4,08123,20100201,16.3000,439.5000,93.159,9.2100,55.3695,53.3400,4.57200,100.00000,...,28.261692,316.440000,0.092100,0.759116,1.803938,2.563054,52.806446,18.982226,349.189706,POINT (-663965.526 55833.920)
5,08001,20200202,34.9663,403.8390,0.000,0.1632,12.5160,10.0584,0.30480,416.66667,...,4.470715,290.764080,0.001632,0.232798,0.285365,0.518162,11.997838,35.067039,9.615961,POINT (-653903.448 -533.739)
6,08031,10200601,2.1211,108.6608,0.000,0.1043,0.7919,8.5344,1.98120,333.33333,...,0.471196,78.235776,0.001043,0.008719,0.030076,0.038795,0.753105,2.256191,12.895092,POINT (-679227.134 2908.884)
7,08031,10100601,1.4296,6.4236,0.000,0.2159,0.5373,15.2400,1.52400,169.44444,...,0.309603,4.624992,0.002159,0.006238,0.019762,0.026000,0.511300,1.564691,12.895092,POINT (-679227.366 2910.126)
8,08059,20100102,0.0014,0.0510,0.000,0.0000,0.0016,7.6200,0.30480,583.33333,...,0.000362,0.036720,0.000000,0.000036,0.000023,0.000060,0.001540,0.101472,9.552287,POINT (-688809.493 7519.325)
9,08059,30290003,0.0400,0.7400,0.000,0.0040,0.0500,18.2880,1.21920,94.44444,...,0.025004,0.532800,0.000040,0.000702,0.001596,0.002298,0.047702,0.484574,42.436624,POINT (-696642.109 5443.609)


In [8]:
gdf_emis.columns

Index(['FIPS', 'SCC', 'VOC', 'NOx', 'NH3', 'SOx', 'PM2_5', 'height', 'diam',
       'temp', 'velocity', 'EIS_ID', 'ghgrp_faci', 'NOX_subpar', 'SO2_subpar',
       'VOC_subpar', 'NH3_subpar', 'PM25FIL_su', 'PM25CON_su', 'PM25_subpa',
       'NOX_reduct', 'SO2_reduct', 'VOC_increa', 'NH3_increa', 'PM25FIL_re',
       'PM25CON_re', 'NOX_out_su', 'SO2_out_su', 'PM25FIL_ou', 'PM25CON_ou',
       'PM25_out_s', 'PM25_reduc', 'VOC_out_su', 'NH3_out_su', 'geometry'],
      dtype='object')

The cobenefits code, which process the air quality emissions based on the CCS technology, generates higher emissions of NOx and SOx for 1-2 facilities, which appears bug in the code. The dataframe below shows two facilities with higher SOx emission by CCS. 

The dataframe below shows the facility with higher NOx emission by CCS. The incorrect emissions are very small (total 0.5 tons for NOx and 4 tons for SOx), and it is unlikely affect our overall results. Nevertheless, these emissions should be fixed from the cobenefits code in future. Otherwise, I need to manually remove these emissions. 

In [21]:
import contextily as ctx
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import os

def plot_spatial_distribution_percent_change_with_basemap(gdf, output_dir, national_scale = False):
    pollutants = ['NH3', 'VOC', 'NOx', 'SOx', 'PM2_5']
    
    # Ensure the GeoDataFrame is in the correct CRS for basemaps
    gdf = gdf.to_crs(epsg=3857)
    
    for pollutant in pollutants:
        # Use gridspec for more precise layout control
        fig = plt.figure(figsize=(10, 6))
        gs = fig.add_gridspec(1, 20)  # Create a 1×20 grid for precise width control
        
        # Main plot takes 18/20 of the width
        ax = fig.add_subplot(gs[0, :19])
        
        col_current = f'{pollutant}'
        col_old = f'{pollutant}_old'
        
        if col_current not in gdf.columns or col_old not in gdf.columns:
            print(f'Columns {col_current} or {col_old} do not exist in the data.')
            continue
        
        # Calculate percent change
        gdf['percent_change'] = ((gdf[col_current] - gdf[col_old]) / gdf[col_old].replace(0, float('nan'))) * 100
        
        # debugging
        gdf[gdf['percent_change'] > 0].to_csv(output_dir + 'debuggin_USA_positive_PM_changes.csv', index=False)
        
        # Set color scale
        vmin, vmax = -100, 100
        if pollutant == 'NH3':
            vmin, vmax = -200, 200
        
        # Plot data
        gdf.plot(column='percent_change', cmap='coolwarm', vmin=vmin, vmax=vmax, 
                legend=False, edgecolor='black', ax=ax, markersize=30, alpha=0.95)
        
        if national_scale:
            # Add basemap
            ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=4)
        else:   
            # Add basemap
            ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=10)
            
        # Set title
        ax.set_title(f'Percent Change in {pollutant} emissions by amine-based CCS')
        
        # Calculate statistics
        total_current = gdf[col_current].sum()
        total_old = gdf[col_old].sum()
        total_percent_change = ((total_current - total_old) / total_old) * 100
        max_percent_change = gdf['percent_change'].max()
        min_percent_change = gdf['percent_change'].min()
        
        # Display statistics
        ax.text(0.5, -0.15, f'Total Percent Change: {total_percent_change:.2f}%\nMax Percent Change: {max_percent_change:.2f}%\nMin Percent Change: {min_percent_change:.2f}%',
                ha='center', va='center', transform=ax.transAxes, fontsize=12, color='black')
        
        # Create colorbar in the last 1/20 columns of the grid
        cbar_ax = fig.add_subplot(gs[0, 19:])
        sm = plt.cm.ScalarMappable(cmap='coolwarm', norm=mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax))
        sm._A = []
        cbar = fig.colorbar(sm, cax=cbar_ax)
        cbar.set_label(f'{pollutant} Percent Change', rotation=270, labelpad=15)
        
        # Minimize spacing between elements
        plt.subplots_adjust(wspace=0.05)  # Very small spacing between map and colorbar
        
        # Save figure
        plt.savefig(os.path.join(output_dir, f'{pollutant}_percent_change_with_basemap.png'), dpi=300, bbox_inches='tight')
        plt.close()

# Example usage
plot_spatial_distribution_percent_change_with_basemap(subset_df, output_dir, national_scale = national_scale_on)


In [5]:
import contextily as ctx
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import os

# Function to plot the percent change of each pollutant and its "_old" version with a basemap
def plot_spatial_distribution_relative_difference_with_basemap(gdf, output_dir, national_scale=False):
    pollutants = ['NH3', 'VOC', 'NOx',  'SOx', 'PM2_5'] #  
    
    # Ensure the GeoDataFrame is in the correct CRS for basemaps (Web Mercator)
    gdf = gdf.to_crs(epsg=3857)

    for pollutant in pollutants:
        # Use gridspec for more precise layout control
        fig = plt.figure(figsize=(10, 6))
        gs = fig.add_gridspec(1, 20)  # Create a 1×20 grid for precise width control

        # Main plot takes 18/20 of the width
        ax = fig.add_subplot(gs[0, :19])
        
        col_current = f'{pollutant}'
        col_old = f'{pollutant}_old'

        # Ensure both the current and "_old" columns exist in the GeoDataFrame
        if col_current not in gdf.columns or col_old not in gdf.columns:
            print(f'Columns {col_current} or {col_old} do not exist in the data.')
            continue
        
        # Calculate the percent change, avoiding division by zero
        gdf['reverse_percent_change'] = ((gdf[col_old] - gdf[col_current]) / gdf[col_current].replace(0, float('nan'))) * 100

        # Plot the spatial distribution of the percent change
        vmin, vmax = -100, 100  # Fixed color scale from -50% to 50%

        if pollutant == 'NH3':
            vmin, vmax = -200, 200 

        gdf.plot(column='reverse_percent_change', cmap='coolwarm', vmin=vmin, vmax=vmax, legend=False, edgecolor='black', 
                 ax=ax, markersize=30, alpha=0.95)  # Increase marker size and reduce transparency

        if national_scale:
            # Add basemap
            ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=4)
        else:   
            # Add basemap
            ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=10)

        ax.set_title(f'Relative Difference in {pollutant} NEI 2020 Emissions Compared to Amine-Based CCS Emissions')

        # Calculate the total, max, and min percent change
        total_current = gdf[col_current].sum()
        total_old = gdf[col_old].sum()
        total_percent_change = ((total_old - total_current) / total_current) * 100
        max_percent_change = gdf['reverse_percent_change'].max()
        min_percent_change = gdf['reverse_percent_change'].min()

        # Display the total, max, and min percent changes on the plot
        ax.text(0.5, -0.15, f'Total Relative Difference: {total_percent_change:.2f}%\nMax Relative Difference: {max_percent_change:.2f}%\nMin Relative Difference: {min_percent_change:.2f}%', 
                ha='center', va='center', transform=ax.transAxes, fontsize=12, color='black')

        # Create colorbar in the last 1/20 columns of the grid
        cbar_ax = fig.add_subplot(gs[0, 19:])
        sm = plt.cm.ScalarMappable(cmap='coolwarm', norm=mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax))
        sm._A = []
        cbar = fig.colorbar(sm, cax=cbar_ax)
        cbar.set_label(f'{pollutant} Relative Difference', rotation=270, labelpad=15)
        
        # Minimize spacing between elements
        plt.subplots_adjust(wspace=0.05)  # Very small spacing between map and colorbar

        # Save the figure for each pollutant as a separate file
        plt.savefig(os.path.join(output_dir, f'{pollutant}_Relative_Difference_with_basemap.png'), dpi=300, bbox_inches='tight')
        plt.close()  # Close the figure to avoid overlapping plots

plot_spatial_distribution_relative_difference_with_basemap(subset_df, output_dir, national_scale = national_scale_on)

To spatial distribution of the emissions changes by amine-based CCS technology are presented here. It shows the percent changes computed using the difference between the emissions with CCS and the original NEI2020 emissions divided by the NEI2020 emissions. For Primary PM2.5, NOx and SOx, it shows negative percent changes because the CCS tech reduces their emissions. For VOC, the percent changes are only slightly positive. 
<table>
  <tr>
    <td><img src="/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/outputs/emissions/CO_CCS/PM2_5_percent_change_with_basemap.png" alt="PM2.5" width="400"/></td>
    <td><img src="/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/outputs/emissions/CO_CCS/VOC_percent_change_with_basemap.png" alt="VOC" width="400"/></td>
  </tr>
  <tr>
    <td><img src="/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/outputs/emissions/CO_CCS/NOx_percent_change_with_basemap.png" alt="NOx" width="400"/></td>
    <td><img src="/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/outputs/emissions/CO_CCS/SOx_percent_change_with_basemap.png" alt="SOx" width="400"/></td>
  </tr>
</table>

The amine-based CCS technology increases NH3 emissions significatly. Several point-source facilities with zero NH3 emissions in the NEI2020 emission inventory have positive NH3 emissions. Since the percent changes calculation used for other pollutants uses the NEI2020 emissions as dinominator, it drops out several facilities with zero NH3 emissions in the NEI2020 inventory. Thus, the percent change calculation used for NH3 is based on the emissions with CCS, instead of the NEI2020 emissions - it computes using the difference between the original NEI2020 emissions and the emissions with CCS, divided by the CCS emissions. The percent changes for other pollutants show how the CCS technology changes their emissions from the NEI2020 inventory while the one for NH3 shows how the NEI2020 inventory changes its emission from the CCS emissions. 
<td><img src="/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/outputs/emissions/CO_CCS/NH3_Relative_Difference_with_basemap.png" alt="NH3" width="800"/></td>

The changes in emissions by the CCS technology is also presented in barplot. It shows total emissions in tonnes by each county (FIPS code) for each pollutant. 

In [3]:
import numpy as np

# sum by FIPS
grouped_df = subset_df.drop(columns='geometry').groupby('EIS_ID').sum().reset_index()

pollutants = [ 'PM2_5' , 'NOx',  'SOx','NH3', 'VOC'] # 
pollutants_old = [f'{pollutant}_old' for pollutant in pollutants]

# Calculate total sums for each pollutant
totals = {pollutant: subset_df[pollutant].sum() for pollutant in pollutants_old}
totals_CCS ={pollutant: subset_df[pollutant].sum() for pollutant in pollutants}

# Plotting
fig, axes = plt.subplots(nrows=len(pollutants), ncols=1, figsize=(20, 20) ) #, sharey=True)
bar_width = 0.35  # Width of the bars
for i, pollutant in enumerate(pollutants):
    ax = axes[i]
    indices = np.arange(len(grouped_df))
    # Plot CCS emissions
    ax.bar(indices, grouped_df[pollutant], bar_width, label=f'{pollutant}')
    
    # Plot NEI2020
    ax.bar(indices + bar_width, grouped_df[pollutants_old[i]], bar_width, label=f'{pollutants_old[i]}')
    
    total_original = totals[f'{pollutant}_old']
    total_new = totals_CCS[f'{pollutant}']
    ax.set_title(f'{pollutant}\nTotal: {total_original:.0f} | Change : {(total_new - total_original):.0f} [tons]', fontsize=20)
    ax.set_xlabel('EIS_ID')
    ax.set_ylabel('Total emissions [tons]')
    ax.set_xticks(indices + bar_width / 2)
    ax.set_xticklabels(grouped_df['EIS_ID'], rotation=90)
    ax.legend(["with CCS tech", "w/o CCS tech"])

plt.tight_layout()
plt.savefig(os.path.join(output_dir, f'Total_Difference.png'), dpi=300, bbox_inches='tight')

plt.show()

KeyError: 'NH3_old'

In [15]:
from IPython.display import display, Markdown

markdown_text = f"The number of facilties applied with the amine-based CCS technology is **{len(subset_df['EIS_ID'].unique())}**, which is all located in the CO state. "
                 
for i, pollutant in enumerate(pollutants):
    total_original = totals[f'{pollutant}_old']
    change = totals_CCS[f'{pollutant}'] - total_original

    # Use Markdown to display the computed value
    markdown_text +=f"For {pollutant}, the total NEI2020 emissions are **{total_original:.0f}** in tons, and it is changed by **{change:.0f}** with amine-based CCS technology. "


# Display the entire paragraph in one go
display(Markdown(markdown_text))

The number of facilties applied with the amine-based CCS technology is **9**, which is all located in the CO state. For NH3, the total NEI2020 emissions are **93** in tons, and it is changed by **2245** with amine-based CCS technology. For VOC, the total NEI2020 emissions are **344** in tons, and it is changed by **24** with amine-based CCS technology. For NOx, the total NEI2020 emissions are **2250** in tons, and it is changed by **-630** with amine-based CCS technology. For SOx, the total NEI2020 emissions are **166** in tons, and it is changed by **-164** with amine-based CCS technology. For PM2_5, the total NEI2020 emissions are **202** in tons, and it is changed by **-193** with amine-based CCS technology. 

# Check a facility emissions from NEI inventory



In [17]:
import geopandas as gpd

# read base and sens emission scenarios
gdf_emis = gpd.read_file('/Users/yunhalee/Documents/LOCAETA/CS_emissions/Colorado_CCS_combined_NEI_point_oilgas_ptegu_ptnonimps.shp')

# Reset index to ensure proper comparison
gdf_emis.reset_index(drop=True, inplace=True)

# Apply CCS emissions to a facility of interest
facility_eis_id = {'Suncor':1099511 } # 'Cherokee':3555811 } 'Landfill': 2001411} # 
                   #'Suncor':1099511, } # 17445711} # Suncor frs_id = '1007923'
		# Cherokee plant alone (facility ID 1007207)  EIS_ID = 17445711
        # NEI emission without landfill facility ID 1007709

species_list = ['NOx', 'SOx', 'PM2_5'] # VOC and NH3 are excluded because the input emissions has NEI emissions for VOC and NH3. 

for key, id in facility_eis_id.items():

    if isinstance(id, int):
        print(f"{id} is integer")
    else:
        print(f"{id} must be integer")
        id = int(id)

    print(f"matching facility: {gdf_emis[gdf_emis['EIS_ID'] == id]}" )

    subset_df = gdf_emis[gdf_emis['EIS_ID'] == id]

subset_df[['PM2_5']].sum()

1099511 is integer
matching facility:          FIPS       SCC  VOC_old   NOx_old  NH3_old  SOx_old  PM2_5_old  \
119718  08001  10200701   8.6938  209.7100      0.0   5.5252    12.0281   
119719  08001  30103202   0.0500    0.9400      0.0  52.9000     0.0700   
119720  08001  30107103   5.2000    0.0000      0.0   0.0000     0.0000   
119721  08001  30600104   2.7600   42.0200      0.0   2.8200     3.9700   
119722  08001  30600106  24.2535  250.6707      0.0  19.3947    26.0017   
...       ...       ...      ...       ...      ...      ...        ...   
119783  08001  40700809   0.4058    0.0000      0.0   0.0000     0.0000   
119784  08001  40704801   0.9300    0.0000      0.0   0.0000     0.0000   
119785  08001  50410310   0.1114    5.8100      0.0   0.0000     0.0000   
119786  08001  50410314   0.0754    2.2600      0.0   0.0100     0.0900   
119787  08001  50410405   0.2900    0.0000      0.0   0.0000     0.0000   

         height     diam        temp  velocity   EIS_ID  ghgr

PM2_5    70.537739
dtype: float64

In [18]:
subset_df[['PM2_5_old']].sum()

PM2_5_old    114.932
dtype: float64

## Debugging NEI emissions (for duplicates)

Date : May 6, 2025 

Whole USA scenario, EIS_ID and SCC no longer a unquie identifier, so there are some duplicates, resulting in incorrect final NEI-CCS emissions. The script below is checking what causes the duplicates. 

In [59]:
import geopandas as gpd
import os

# LA_CCS and CO_CCS don't have the duplicates case for EIS_ID and SCC

# read base and sens emission scenarios
gdf_debug = gpd.read_file(
#  '/Users/yunhalee/Documents/LOCAETA/RCM/INMAP/evaldata_v1.6.1/2020_nei_emissions/combined_NEI2020_pt_oilgas_ptegu_ptnonipm.shp')
 '/Users/yunhalee/Documents/LOCAETA/CS_emissions/USA_point_CCS.shp')

debug_output_dir = '/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/outputs/emissions/USA_CCS/'

# Reset index to ensure proper comparison
gdf_debug.reset_index(drop=True, inplace=True)

#print(gdf_debug[(gdf_debug['EIS_ID'].astype(int) == 715711) & (gdf_debug['SCC'].astype(int) == 10100601)]) 
#print(gdf_debug[(gdf_debug['EIS_ID'].astype(int) == 12611) & (gdf_debug['SCC'].astype(int) == 10200602)])  
print(gdf_debug[(gdf_debug['EIS_ID'].astype(int) == 6011511) & (gdf_debug['SCC'].astype(int) == 10100504)])   

# ptnonipm_2 has PM2_5 0.99218
# ptegu_1 has PM2_5 9.899564

       FIPS       SCC       VOC       NOx     NH3       SOx     PM2_5  \
1958  24017  10100504  0.008430  13.59140  1.2094  7.594300  0.194396   
1959  24017  10100504  0.001845   3.97610  0.0000  0.091285  0.042946   
1960  24017  10100504  0.048585   5.84995  0.0000  0.692435  0.355238   

       height    diam       temp  ...  PM25CON_re  NOX_out_su  SO2_out_su  \
1958  14.6304  6.7818  472.22222  ...    0.398802   16.860564     0.08378   
1959  14.6304  6.7818  472.22222  ...    0.398802   16.860564     0.08378   
1960  56.6928  1.5240  152.77778  ...    0.398802   16.860564     0.08378   

      PM25FIL_ou  PM25CON_ou  PM25_out_s  PM25_reduc  VOC_out_su  NH3_out_su  \
1958     0.00505    0.025455    0.030505    0.562074    2.964378  278.554295   
1959     0.00505    0.025455    0.030505    0.562074    2.964378  278.554295   
1960     0.00505    0.025455    0.030505    0.562074    2.964378  278.554295   

                          geometry  
1958  POINT (1722420.103 9071.085)  
195

In [60]:
# Identify duplicates
duplicate_keys = (
    gdf_debug.groupby(['EIS_ID', 'SCC'])
    .size()
    .reset_index(name='count')
    .query('count > 1')[['EIS_ID', 'SCC']]
)
duplicates = gdf_debug.merge(duplicate_keys, on=['EIS_ID', 'SCC'], how='inner')
duplicates['row_key'] = duplicates.index  # Track original index

# Case 1: Same EIS_ID, SCC, ghgrp_faci but different PM2_5
case1_keys = (
    duplicates.groupby(['EIS_ID', 'SCC', 'ghgrp_faci'])['PM2_5']
    .nunique()
    .reset_index(name='pm25_variety')
    .query('pm25_variety > 1')[['EIS_ID', 'SCC', 'ghgrp_faci']]
)
case1 = duplicates.merge(case1_keys, on=['EIS_ID', 'SCC', 'ghgrp_faci'])
case1_row_keys = set(case1['row_key'])

# Exclude Case 1 rows before doing Case 2
case_others = duplicates[~duplicates['row_key'].isin(case1_row_keys)]

# Case 2: Multiple ghgrp_faci for the same NEI (EIS_ID + SCC)
case2_keys = (
    case_others.groupby(['EIS_ID', 'SCC'])['ghgrp_faci']
    .nunique()
    .reset_index(name='ghgrp_faci_count')
    .query('ghgrp_faci_count > 1')[['EIS_ID', 'SCC']]
)
case2 = case_others.merge(case2_keys, on=['EIS_ID', 'SCC'])
case2_row_keys = set(case2['row_key'])

# Case 3: Remaining (All IDs are same but two subparts, C and D, results in different NH3/VOC increase)
remaining_row_keys = set(duplicates['row_key']) - case1_row_keys - case2_row_keys
case3 = duplicates[duplicates['row_key'].isin(remaining_row_keys)]

# Output
print(f"Total Duplicates: {len(duplicates)} rows")
print(f"Case 1: {len(case1)} rows")
print(f"Case 2: {len(case2)} rows")
print(f"Case 3: {len(case3)} rows")
print(f"Sum of all cases: {len(case1) + len(case2) + len(case3)} rows")


case1.to_csv(debug_output_dir + 'Case1_one_ghgrp_fac_multiple_NEI_duplicates.csv', index=False)
case2.to_csv(debug_output_dir + 'Case2_more_than_one_ghgrp_faci_per_NEI_duplicates.csv', index=False)
case3.to_csv(debug_output_dir + 'Case3_other_duplicates.csv', index=False)

Total Duplicates: 3344 rows
Case 1: 946 rows
Case 2: 2218 rows
Case 3: 180 rows
Sum of all cases: 3344 rows


In [61]:
3344/2

1672.0